In [51]:
#import nltk
#nltk.download()#下载Model里的第一个包Averaged Perceptron Tagger

In [52]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import string
import copy
import json
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
data = pd.read_csv('News_Final.csv')[0:1000] #导入csv文件
samples = dict(data['Headline'])#提取Headline列进行特征提取
samples

{0: 'Obama Lays Wreath at Arlington National Cemetery. President Barack Obama has laid a wreath at the Tomb of the Unknowns to honor',
 1: 'Tim Haywood, investment director business-unit head for fixed income at Gam, discusses the China beige book and the state of the economy.',
 2: "Nouriel Roubini, NYU professor and chairman at Roubini Global Economics, explains why the global economy isn't facing the same conditions",
 3: "Finland's economy expanded marginally in the three months ended December, after contracting in the previous quarter, preliminary figures from Statistics Finland showed Monday. ",
 4: 'Tourism and public spending continued to boost the economy in January, in light of contraction in private consumption and exports, according to the Bank of Thailand data. ',
 5: 'Over 100 attendees expected to see latest version of Microsoft Dynamics SL and Dynamics GP (PRWeb February 29, 2016) Read the full story at http://www.prweb.com/releases/2016/03/prweb13238571.htm ',
 6: 'RAM

In [53]:
words_list = []
for item in samples:
    sentence = list((str(samples[item])).lower())#将字母转化为小写
    for i in range(len(sentence)):
        if (not sentence[i].isalnum()) and sentence[i] != '\'':#用空格替换非字母
            sentence[i] = ' '
    sen = ''
    for i in sentence:
        sen += str(i)
    words_list += [list(sen.split())]
words_list

[['obama',
  'lays',
  'wreath',
  'at',
  'arlington',
  'national',
  'cemetery',
  'president',
  'barack',
  'obama',
  'has',
  'laid',
  'a',
  'wreath',
  'at',
  'the',
  'tomb',
  'of',
  'the',
  'unknowns',
  'to',
  'honor'],
 ['tim',
  'haywood',
  'investment',
  'director',
  'business',
  'unit',
  'head',
  'for',
  'fixed',
  'income',
  'at',
  'gam',
  'discusses',
  'the',
  'china',
  'beige',
  'book',
  'and',
  'the',
  'state',
  'of',
  'the',
  'economy'],
 ['nouriel',
  'roubini',
  'nyu',
  'professor',
  'and',
  'chairman',
  'at',
  'roubini',
  'global',
  'economics',
  'explains',
  'why',
  'the',
  'global',
  'economy',
  "isn't",
  'facing',
  'the',
  'same',
  'conditions'],
 ["finland's",
  'economy',
  'expanded',
  'marginally',
  'in',
  'the',
  'three',
  'months',
  'ended',
  'december',
  'after',
  'contracting',
  'in',
  'the',
  'previous',
  'quarter',
  'preliminary',
  'figures',
  'from',
  'statistics',
  'finland',
  'showed'

In [54]:
#读取停用词
with open('stopwords.txt','r') as f:
    stopwords = set(f.read().split())
print(stopwords)

{'by', 't', 'us', 'we', 'after', "we've", 'then', "they'd", 'were', 'ought', 'would', 'how', 'be', 'own', 'doing', 'it', 'some', 'why', "you'd", 'ourselves', "here's", 'through', "i'm", 'above', 'more', 'themselves', 'm', 'yourselves', "he's", 'cannot', 'him', 'if', 'h', 'than', 'while', 'off', 'our', 'there', 'you', 'do', 'who', "he'd", 'your', "they're", 'must', 'both', 'was', 'few', 'not', 'below', 'me', 'other', 'x', 'been', "we're", 'about', 'most', 'over', 'their', 'those', 'for', "you've", 'but', 'does', 'itself', 'until', "wasn't", 'this', 'v', 'ours', 'no', 'to', 'what', 'each', 'a', 'at', 'up', 'during', 'an', 'before', 'being', 'her', 'onto', "she'd", 'should', 'z', "hasn't", "weren't", "mustn't", 'again', 'r', 'further', "they've", "we'll", "can't", "you'll", 'they', "hadn't", "don't", "that's", 'yours', 'did', "i've", 'had', 'can', 'nor', "aren't", 'my', 'upon', "they'll", "she'll", 'of', "isn't", "he'll", 'is', "who's", 'she', 'because', 'and', "when's", 'which', "what's"

In [55]:
#筛选非停用词
for  i in range(len(words_list)):
    new = []
    for j in words_list[i]:
        if j not in stopwords:
            new.append(j)
    words_list[i] = new
#处理一些异常字符
for i in range(len(words_list)):
    ls = words_list[i]
    for j in range(len(ls)):
        if ls[j][0] == '\'':
            if ls[j][-1] == '\'':
                ls[j] = ls[j][1:-1]
                continue
        if '\'' in ls[j]:
            ls[j] = ls[j][0:ls[j].index('\'')]
    words_list[i] = ls
words_list

[['obama',
  'lays',
  'wreath',
  'arlington',
  'national',
  'cemetery',
  'president',
  'barack',
  'obama',
  'laid',
  'wreath',
  'tomb',
  'unknowns',
  'honor'],
 ['tim',
  'haywood',
  'investment',
  'director',
  'business',
  'unit',
  'head',
  'fixed',
  'income',
  'gam',
  'discusses',
  'china',
  'beige',
  'book',
  'state',
  'economy'],
 ['nouriel',
  'roubini',
  'nyu',
  'professor',
  'chairman',
  'roubini',
  'global',
  'economics',
  'explains',
  'global',
  'economy',
  'facing',
  'conditions'],
 ['finland',
  'economy',
  'expanded',
  'marginally',
  'three',
  'months',
  'ended',
  'december',
  'contracting',
  'previous',
  'quarter',
  'preliminary',
  'figures',
  'statistics',
  'finland',
  'showed',
  'monday'],
 ['tourism',
  'public',
  'spending',
  'continued',
  'boost',
  'economy',
  'january',
  'light',
  'contraction',
  'private',
  'consumption',
  'exports',
  'according',
  'bank',
  'thailand',
  'data'],
 ['100',
  'attendees'

In [56]:
#get_wordnet_pos函数是获取单词的词性
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
#生成词形还原对象
wordnet_lemmatizer = WordNetLemmatizer()

In [57]:
#wordnet.synsets('roubini')#在词典里查找单词

In [58]:
#做词形还原处理
for item in words_list:
    for i in range(len(item)):
        try:
            item[i] = wordnet_lemmatizer.lemmatize(item[i],pos=get_wordnet_pos(pos_tag([item[i]])[0][1]))
        except:
            pass

In [59]:
words_list

[['obama',
  'lay',
  'wreath',
  'arlington',
  'national',
  'cemetery',
  'president',
  'barack',
  'obama',
  'laid',
  'wreath',
  'tomb',
  'unknown',
  'honor'],
 ['tim',
  'haywood',
  'investment',
  'director',
  'business',
  'unit',
  'head',
  'fix',
  'income',
  'gam',
  'discus',
  'china',
  'beige',
  'book',
  'state',
  'economy'],
 ['nouriel',
  'roubini',
  'nyu',
  'professor',
  'chairman',
  'roubini',
  'global',
  'economics',
  'explains',
  'global',
  'economy',
  'face',
  'condition'],
 ['finland',
  'economy',
  'expand',
  'marginally',
  'three',
  'month',
  'end',
  'december',
  'contract',
  'previous',
  'quarter',
  'preliminary',
  'figure',
  'statistic',
  'finland',
  'show',
  'monday'],
 ['tourism',
  'public',
  'spending',
  'continued',
  'boost',
  'economy',
  'january',
  'light',
  'contraction',
  'private',
  'consumption',
  'export',
  'accord',
  'bank',
  'thailand',
  'data'],
 ['100',
  'attendee',
  'expect',
  'see',
  'l

In [60]:
#处理某个异常词
for item in words_list:
    if "'slightly'" in item:
        print(item)
        item[item.index("'slightly'")] = 'slightly'
    if '' in item:
        print(item)
        item.remove('')

['ever', 'since', '', 'mac', 'tv', 'advertising', 'campaign', 'mid', '00s', 'latter', 'two', 'tech', 'giant', 'engage', 'heat', 'ad', 'base', 'conflict', 'late', 'round', 'tv', 'spot', 'anything', 'go', 'go', 'change', 'time', 'soon', 'late', 'advertisement', '30', 'second', 'less', 'produce', 'coincide', 'coverage', 'oscar', 'award', 'ceremony', 'feature', 'two', 'woman', 'summarise']
['fed', 'dudley', 'risk', 'economy', 'tilt', "'slightly'", 'downside']
['microsoft', 'continue', 'drive', 'make', 'hardware', 'lumia', 'microsoft', 'achieve', '', 'surface', 'pro']
['window', '10', 'free', 'communicate', 'message', 'mass', 'easily', 'microsoft', 'msft', '0', '00', '', 'big', 'success', 'last', 'year']
['land', 'tel', 'aviv', 'plane', 'captain', 'announces', 'land', '', 'palestine', 'announce', 'prepare', 'land', 'quot', 'palestine', 'quot']
['press', 'clipping', '1937', '', 'arab', 'celebrate', 'saeb', 'erekat', 'say', 'recently', 'palestinian', 'effort', 'nazi', 'regime']
['less', 'mont

In [61]:
for item in words_list:
    if '' in item:
        print(item)
        item.remove('')

['now', 'look', 'see', 'improve', 'consumer', 'confidence', 'translate', 'strong', 'christmas', 'sale', '', 'photograph', 'brendon']
['beat', 'buy', '', 'microsoft', 'say', 'thursday', 'purchase', 'mobile', 'data', 'lab', 'top', 'gross', 'finance', 'app', 'itunes', 'store']
['economy', 'hurt', 'public', 'spectacle', 'execution', 'april', 'help', '', 'photograph', 'ulet', 'ifansasti', 'getty', 'image']


In [62]:
for item in words_list:
    if '' in item:
        print(item)
        item.remove('')

In [63]:
#收集所有词
words=[]
for i in words_list:
    words+=i
words=sorted(list(set(words)))

In [64]:
words

['0',
 '00',
 '000',
 '00s',
 '01',
 '03',
 '039',
 '07',
 '08',
 '09',
 '1',
 '10',
 '100',
 '1000',
 '100m',
 '103',
 '104',
 '104665',
 '108',
 '10th',
 '11',
 '110',
 '12',
 '12th',
 '13',
 '13th',
 '14',
 '15',
 '150',
 '16',
 '17',
 '175',
 '18',
 '185',
 '19',
 '1917',
 '1937',
 '1948',
 '1953',
 '1977',
 '1980s',
 '199',
 '1995',
 '1997',
 '1q15',
 '1q16',
 '1tb',
 '2',
 '20',
 '200',
 '2000',
 '2003',
 '2008',
 '200gb',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2020',
 '20th',
 '21',
 '22',
 '22nd',
 '24',
 '249',
 '25',
 '250',
 '26',
 '260',
 '27',
 '271',
 '28',
 '29',
 '3',
 '30',
 '300',
 '3000',
 '30th',
 '31',
 '32',
 '322',
 '36',
 '360',
 '365',
 '37',
 '38',
 '3d',
 '3gs',
 '4',
 '400',
 '434',
 '5',
 '50',
 '500',
 '50mn',
 '51',
 '53',
 '54',
 '57',
 '595',
 '6',
 '600',
 '602',
 '62',
 '65',
 '67',
 '68',
 '7',
 '70',
 '71',
 '73',
 '75',
 '75tb',
 '77',
 '78',
 '7m',
 '7th',
 '8',
 '80',
 '800',
 '810g',
 '83',
 '84th',
 '

In [65]:
#生成单词索引
words_dict = {}
for i in range(len(words)):
    words_dict[words[i]] = i

In [66]:
print(words_dict)

{'0': 0, '00': 1, '000': 2, '00s': 3, '01': 4, '03': 5, '039': 6, '07': 7, '08': 8, '09': 9, '1': 10, '10': 11, '100': 12, '1000': 13, '100m': 14, '103': 15, '104': 16, '104665': 17, '108': 18, '10th': 19, '11': 20, '110': 21, '12': 22, '12th': 23, '13': 24, '13th': 25, '14': 26, '15': 27, '150': 28, '16': 29, '17': 30, '175': 31, '18': 32, '185': 33, '19': 34, '1917': 35, '1937': 36, '1948': 37, '1953': 38, '1977': 39, '1980s': 40, '199': 41, '1995': 42, '1997': 43, '1q15': 44, '1q16': 45, '1tb': 46, '2': 47, '20': 48, '200': 49, '2000': 50, '2003': 51, '2008': 52, '200gb': 53, '2010': 54, '2011': 55, '2012': 56, '2013': 57, '2014': 58, '2015': 59, '2016': 60, '2017': 61, '2018': 62, '2020': 63, '20th': 64, '21': 65, '22': 66, '22nd': 67, '24': 68, '249': 69, '25': 70, '250': 71, '26': 72, '260': 73, '27': 74, '271': 75, '28': 76, '29': 77, '3': 78, '30': 79, '300': 80, '3000': 81, '30th': 82, '31': 83, '32': 84, '322': 85, '36': 86, '360': 87, '365': 88, '37': 89, '38': 90, '3d': 91,

In [67]:
#格式为id:词语
ids_dict={}
for i in words_dict:
    ids_dict[words_dict[i]]=i

In [68]:
ids_dict

{0: '0',
 1: '00',
 2: '000',
 3: '00s',
 4: '01',
 5: '03',
 6: '039',
 7: '07',
 8: '08',
 9: '09',
 10: '1',
 11: '10',
 12: '100',
 13: '1000',
 14: '100m',
 15: '103',
 16: '104',
 17: '104665',
 18: '108',
 19: '10th',
 20: '11',
 21: '110',
 22: '12',
 23: '12th',
 24: '13',
 25: '13th',
 26: '14',
 27: '15',
 28: '150',
 29: '16',
 30: '17',
 31: '175',
 32: '18',
 33: '185',
 34: '19',
 35: '1917',
 36: '1937',
 37: '1948',
 38: '1953',
 39: '1977',
 40: '1980s',
 41: '199',
 42: '1995',
 43: '1997',
 44: '1q15',
 45: '1q16',
 46: '1tb',
 47: '2',
 48: '20',
 49: '200',
 50: '2000',
 51: '2003',
 52: '2008',
 53: '200gb',
 54: '2010',
 55: '2011',
 56: '2012',
 57: '2013',
 58: '2014',
 59: '2015',
 60: '2016',
 61: '2017',
 62: '2018',
 63: '2020',
 64: '20th',
 65: '21',
 66: '22',
 67: '22nd',
 68: '24',
 69: '249',
 70: '25',
 71: '250',
 72: '26',
 73: '260',
 74: '27',
 75: '271',
 76: '28',
 77: '29',
 78: '3',
 79: '30',
 80: '300',
 81: '3000',
 82: '30th',
 83: '31',

In [69]:
find_words=list(map(set,words_list))#建立每篇文章的词语集，用集合查找加快速度

In [70]:
#统计每个单词的文档频率df
words_df = {}
for word in words_dict:
    num = 0
    for sentence in find_words:
        if word in sentence:
            num += 1
    words_df[words_dict[word]] = num

In [71]:
words_df

{0: 30,
 1: 2,
 2: 34,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 33,
 11: 53,
 12: 12,
 13: 2,
 14: 1,
 15: 2,
 16: 2,
 17: 1,
 18: 2,
 19: 2,
 20: 3,
 21: 3,
 22: 15,
 23: 2,
 24: 1,
 25: 2,
 26: 3,
 27: 3,
 28: 5,
 29: 3,
 30: 5,
 31: 1,
 32: 8,
 33: 1,
 34: 7,
 35: 1,
 36: 1,
 37: 2,
 38: 1,
 39: 1,
 40: 1,
 41: 1,
 42: 1,
 43: 1,
 44: 1,
 45: 3,
 46: 4,
 47: 21,
 48: 3,
 49: 2,
 50: 3,
 51: 1,
 52: 2,
 53: 2,
 54: 3,
 55: 1,
 56: 1,
 57: 2,
 58: 8,
 59: 41,
 60: 30,
 61: 4,
 62: 11,
 63: 1,
 64: 1,
 65: 6,
 66: 1,
 67: 1,
 68: 6,
 69: 1,
 70: 1,
 71: 2,
 72: 4,
 73: 1,
 74: 1,
 75: 8,
 76: 2,
 77: 10,
 78: 41,
 79: 21,
 80: 4,
 81: 3,
 82: 4,
 83: 3,
 84: 1,
 85: 1,
 86: 1,
 87: 4,
 88: 15,
 89: 1,
 90: 2,
 91: 1,
 92: 1,
 93: 26,
 94: 1,
 95: 2,
 96: 34,
 97: 6,
 98: 1,
 99: 1,
 100: 1,
 101: 2,
 102: 2,
 103: 1,
 104: 1,
 105: 24,
 106: 2,
 107: 2,
 108: 1,
 109: 1,
 110: 1,
 111: 1,
 112: 14,
 113: 1,
 114: 1,
 115: 1,
 116: 1,
 117: 1,
 118: 2,
 119: 3,
 120: 1,
 12

In [72]:
#剔除df小于5的单词
select=[]
for word in words_df:
    if words_df[word]<=5:
        select.append(word)

In [73]:
select

[1,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 61,
 63,
 64,
 66,
 67,
 69,
 70,
 71,
 72,
 73,
 74,
 76,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 89,
 90,
 91,
 92,
 94,
 95,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 106,
 107,
 108,
 109,
 110,
 111,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 142,
 143,
 144,
 145,
 146,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 161,
 162,
 163,
 164,
 166,
 167,
 168,
 169,
 170,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 182,
 183,
 185,
 186,
 187,
 188,
 191,
 192,
 193,
 195,
 196,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,

In [74]:
for word in select:
    words_df.pop(word)

In [75]:
words_df

{0: 30,
 2: 34,
 10: 33,
 11: 53,
 12: 12,
 22: 15,
 32: 8,
 34: 7,
 47: 21,
 58: 8,
 59: 41,
 60: 30,
 62: 11,
 65: 6,
 68: 6,
 75: 8,
 77: 10,
 78: 41,
 79: 21,
 88: 15,
 93: 26,
 96: 34,
 97: 6,
 105: 24,
 112: 14,
 122: 14,
 130: 19,
 141: 8,
 147: 9,
 160: 10,
 165: 6,
 171: 36,
 181: 10,
 184: 12,
 189: 7,
 190: 7,
 194: 7,
 197: 22,
 235: 7,
 240: 16,
 241: 8,
 242: 8,
 243: 8,
 247: 8,
 253: 14,
 270: 15,
 273: 6,
 275: 10,
 278: 8,
 279: 24,
 281: 11,
 286: 8,
 293: 7,
 295: 6,
 297: 8,
 299: 8,
 300: 9,
 305: 11,
 308: 8,
 317: 56,
 322: 20,
 323: 13,
 333: 10,
 338: 17,
 342: 16,
 345: 30,
 347: 9,
 356: 14,
 360: 45,
 364: 9,
 371: 6,
 373: 11,
 382: 6,
 384: 6,
 387: 10,
 413: 7,
 422: 20,
 427: 10,
 435: 16,
 436: 6,
 444: 9,
 448: 13,
 451: 27,
 457: 6,
 467: 9,
 469: 44,
 473: 10,
 479: 15,
 494: 11,
 498: 12,
 500: 7,
 502: 10,
 506: 8,
 512: 6,
 517: 10,
 531: 32,
 535: 16,
 547: 10,
 558: 14,
 566: 24,
 567: 7,
 569: 12,
 584: 10,
 599: 12,
 609: 6,
 618: 7,
 621: 6,

In [76]:
words=[]
for i in words_df:
    words.append(ids_dict[i])

In [77]:
words

['0',
 '000',
 '1',
 '10',
 '100',
 '12',
 '18',
 '19',
 '2',
 '2014',
 '2015',
 '2016',
 '2018',
 '21',
 '24',
 '271',
 '29',
 '3',
 '30',
 '365',
 '4',
 '5',
 '50',
 '6',
 '7',
 '8',
 '9',
 'abbas',
 'able',
 'accelerate',
 'access',
 'accord',
 'acquire',
 'across',
 'activist',
 'activity',
 'ad',
 'add',
 'agency',
 'ago',
 'agree',
 'agreement',
 'ahead',
 'aim',
 'al',
 'allow',
 'almost',
 'along',
 'already',
 'also',
 'although',
 'amazon',
 'american',
 'amid',
 'amman',
 'among',
 'amount',
 'analyst',
 'android',
 'announce',
 'annual',
 'another',
 'ap',
 'app',
 'appear',
 'apple',
 'application',
 'apps',
 'arabia',
 'area',
 'arm',
 'around',
 'arun',
 'asia',
 'ask',
 'attempt',
 'augment',
 'authority',
 'available',
 'avenue',
 'away',
 'azure',
 'back',
 'bad',
 'band',
 'bank',
 'barack',
 'base',
 'become',
 'begin',
 'behind',
 'beijing',
 'believe',
 'benjamin',
 'best',
 'big',
 'billion',
 'block',
 'body',
 'book',
 'boom',
 'boost',
 'brand',
 'bring',
 'br

In [78]:
#生成单词索引
Words_Dict = {}
for i in range(len(words)):
    Words_Dict[words[i]] = i

In [79]:
Words_Dict

{'0': 0,
 '000': 1,
 '1': 2,
 '10': 3,
 '100': 4,
 '12': 5,
 '18': 6,
 '19': 7,
 '2': 8,
 '2014': 9,
 '2015': 10,
 '2016': 11,
 '2018': 12,
 '21': 13,
 '24': 14,
 '271': 15,
 '29': 16,
 '3': 17,
 '30': 18,
 '365': 19,
 '4': 20,
 '5': 21,
 '50': 22,
 '6': 23,
 '7': 24,
 '8': 25,
 '9': 26,
 'abbas': 27,
 'able': 28,
 'accelerate': 29,
 'access': 30,
 'accord': 31,
 'acquire': 32,
 'across': 33,
 'activist': 34,
 'activity': 35,
 'ad': 36,
 'add': 37,
 'agency': 38,
 'ago': 39,
 'agree': 40,
 'agreement': 41,
 'ahead': 42,
 'aim': 43,
 'al': 44,
 'allow': 45,
 'almost': 46,
 'along': 47,
 'already': 48,
 'also': 49,
 'although': 50,
 'amazon': 51,
 'american': 52,
 'amid': 53,
 'amman': 54,
 'among': 55,
 'amount': 56,
 'analyst': 57,
 'android': 58,
 'announce': 59,
 'annual': 60,
 'another': 61,
 'ap': 62,
 'app': 63,
 'appear': 64,
 'apple': 65,
 'application': 66,
 'apps': 67,
 'arabia': 68,
 'area': 69,
 'arm': 70,
 'around': 71,
 'arun': 72,
 'asia': 73,
 'ask': 74,
 'attempt': 75,


In [80]:
#统计每个单词的文档频率df
Words_DF = {}
for word in words_df:
    Words_DF[Words_Dict[ids_dict[word]]]=words_df[word]

In [81]:
Words_DF

{0: 30,
 1: 34,
 2: 33,
 3: 53,
 4: 12,
 5: 15,
 6: 8,
 7: 7,
 8: 21,
 9: 8,
 10: 41,
 11: 30,
 12: 11,
 13: 6,
 14: 6,
 15: 8,
 16: 10,
 17: 41,
 18: 21,
 19: 15,
 20: 26,
 21: 34,
 22: 6,
 23: 24,
 24: 14,
 25: 14,
 26: 19,
 27: 8,
 28: 9,
 29: 10,
 30: 6,
 31: 36,
 32: 10,
 33: 12,
 34: 7,
 35: 7,
 36: 7,
 37: 22,
 38: 7,
 39: 16,
 40: 8,
 41: 8,
 42: 8,
 43: 8,
 44: 14,
 45: 15,
 46: 6,
 47: 10,
 48: 8,
 49: 24,
 50: 11,
 51: 8,
 52: 7,
 53: 6,
 54: 8,
 55: 8,
 56: 9,
 57: 11,
 58: 8,
 59: 56,
 60: 20,
 61: 13,
 62: 10,
 63: 17,
 64: 16,
 65: 30,
 66: 9,
 67: 14,
 68: 45,
 69: 9,
 70: 6,
 71: 11,
 72: 6,
 73: 6,
 74: 10,
 75: 7,
 76: 20,
 77: 10,
 78: 16,
 79: 6,
 80: 9,
 81: 13,
 82: 27,
 83: 6,
 84: 9,
 85: 44,
 86: 10,
 87: 15,
 88: 11,
 89: 12,
 90: 7,
 91: 10,
 92: 8,
 93: 6,
 94: 10,
 95: 32,
 96: 16,
 97: 10,
 98: 14,
 99: 24,
 100: 7,
 101: 12,
 102: 10,
 103: 12,
 104: 6,
 105: 7,
 106: 6,
 107: 8,
 108: 6,
 109: 6,
 110: 7,
 111: 41,
 112: 12,
 113: 22,
 114: 9,
 115: 7,


In [82]:
#格式为ID:词语
IDs_Dict={}
for i in Words_Dict:
    IDs_Dict[Words_Dict[i]]=i

In [83]:
IDs_Dict

{0: '0',
 1: '000',
 2: '1',
 3: '10',
 4: '100',
 5: '12',
 6: '18',
 7: '19',
 8: '2',
 9: '2014',
 10: '2015',
 11: '2016',
 12: '2018',
 13: '21',
 14: '24',
 15: '271',
 16: '29',
 17: '3',
 18: '30',
 19: '365',
 20: '4',
 21: '5',
 22: '50',
 23: '6',
 24: '7',
 25: '8',
 26: '9',
 27: 'abbas',
 28: 'able',
 29: 'accelerate',
 30: 'access',
 31: 'accord',
 32: 'acquire',
 33: 'across',
 34: 'activist',
 35: 'activity',
 36: 'ad',
 37: 'add',
 38: 'agency',
 39: 'ago',
 40: 'agree',
 41: 'agreement',
 42: 'ahead',
 43: 'aim',
 44: 'al',
 45: 'allow',
 46: 'almost',
 47: 'along',
 48: 'already',
 49: 'also',
 50: 'although',
 51: 'amazon',
 52: 'american',
 53: 'amid',
 54: 'amman',
 55: 'among',
 56: 'amount',
 57: 'analyst',
 58: 'android',
 59: 'announce',
 60: 'annual',
 61: 'another',
 62: 'ap',
 63: 'app',
 64: 'appear',
 65: 'apple',
 66: 'application',
 67: 'apps',
 68: 'arabia',
 69: 'area',
 70: 'arm',
 71: 'around',
 72: 'arun',
 73: 'asia',
 74: 'ask',
 75: 'attempt',


In [84]:
words_set=set(words)

In [85]:
words_set

{'0',
 '000',
 '1',
 '10',
 '100',
 '12',
 '18',
 '19',
 '2',
 '2014',
 '2015',
 '2016',
 '2018',
 '21',
 '24',
 '271',
 '29',
 '3',
 '30',
 '365',
 '4',
 '5',
 '50',
 '6',
 '7',
 '8',
 '9',
 'abbas',
 'able',
 'accelerate',
 'access',
 'accord',
 'acquire',
 'across',
 'activist',
 'activity',
 'ad',
 'add',
 'agency',
 'ago',
 'agree',
 'agreement',
 'ahead',
 'aim',
 'al',
 'allow',
 'almost',
 'along',
 'already',
 'also',
 'although',
 'amazon',
 'american',
 'amid',
 'amman',
 'among',
 'amount',
 'analyst',
 'android',
 'announce',
 'annual',
 'another',
 'ap',
 'app',
 'appear',
 'apple',
 'application',
 'apps',
 'arabia',
 'area',
 'arm',
 'around',
 'arun',
 'asia',
 'ask',
 'attempt',
 'augment',
 'authority',
 'available',
 'avenue',
 'away',
 'azure',
 'back',
 'bad',
 'band',
 'bank',
 'barack',
 'base',
 'become',
 'begin',
 'behind',
 'beijing',
 'believe',
 'benjamin',
 'best',
 'big',
 'billion',
 'block',
 'body',
 'book',
 'boom',
 'boost',
 'brand',
 'bring',
 'br

In [86]:
Words_List=[]
for i in range(len(words_list)):
    new=[]
    for j in words_list[i]:
        if j in  words_set:
            new.append(j)
    Words_List.append(new)

In [87]:
Words_List

[['obama', 'national', 'president', 'barack', 'obama'],
 ['investment',
  'business',
  'head',
  'income',
  'china',
  'book',
  'state',
  'economy'],
 ['global', 'global', 'economy', 'face'],
 ['economy',
  'expand',
  'three',
  'month',
  'end',
  'december',
  'contract',
  'previous',
  'quarter',
  'figure',
  'statistic',
  'show',
  'monday'],
 ['public',
  'spending',
  'boost',
  'economy',
  'export',
  'accord',
  'bank',
  'data'],
 ['100',
  'expect',
  'see',
  'late',
  'version',
  'microsoft',
  'february',
  '29',
  '2016',
  'read',
  'full',
  'story',
  'com',
  'release',
  '2016'],
 ['ramallah',
  'february',
  '2016',
  'palestine',
  'organization',
  'general',
  'thursday',
  'concern',
  'president',
  'visit',
  'jerusalem',
  'jordan'],
 ['first',
  'obama',
  'state',
  'room',
  'white',
  'house',
  'washington',
  'wednesday',
  'feb',
  '24',
  '2016',
  'student',
  'student',
  '10',
  'school',
  'community',
  'organization',
  'across',
  'co

In [88]:
#将单词替换为代号
words_copy = Words_List
for sentence in words_copy:
    for i in range(len(sentence)):
        sentence[i] = Words_Dict[sentence[i]]

In [89]:
words_copy

[[446, 429, 508, 86, 446],
 [334, 111, 296, 321, 124, 99, 627, 196],
 [278, 278, 196, 226],
 [196, 220, 667, 421, 204, 168, 145, 511, 529, 242, 629, 598, 418],
 [522, 619, 101, 196, 225, 31, 85, 163],
 [4, 221, 584, 362, 697, 407, 235, 16, 11, 538, 268, 636, 130, 554, 11],
 [534, 235, 11, 473, 469, 273, 668, 137, 508, 702, 343, 345],
 [249,
  446,
  627,
  571,
  715,
  311,
  705,
  711,
  234,
  14,
  11,
  640,
  640,
  3,
  579,
  133,
  469,
  33,
  152,
  476,
  532,
  715,
  311,
  532,
  588],
 [394, 384, 190, 418, 422, 4, 725],
 [435,
  172,
  234,
  16,
  659,
  276,
  407,
  656,
  567,
  65,
  435,
  588,
  36,
  254,
  716,
  3,
  287,
  106,
  716],
 [407,
  460,
  631,
  108,
  156,
  107,
  548,
  456,
  518,
  369,
  58,
  178,
  67,
  716,
  3,
  518,
  585,
  425,
  352,
  228,
  410,
  536,
  144,
  539],
 [494,
  82,
  681,
  80,
  725,
  718,
  275,
  425,
  58,
  407,
  279,
  437,
  95,
  494,
  460,
  407,
  384],
 [196,
  220,
  0,
  2,
  486,
  361,
  725,
  

In [90]:
#计算每篇文章的词频
from collections import Counter
Words_TF = {}
for i in range(len(words_copy)):
    Words_TF[i] = dict(Counter(words_copy[i]))

In [91]:
Words_TF

{0: {86: 1, 429: 1, 446: 2, 508: 1},
 1: {99: 1, 111: 1, 124: 1, 196: 1, 296: 1, 321: 1, 334: 1, 627: 1},
 2: {196: 1, 226: 1, 278: 2},
 3: {145: 1,
  168: 1,
  196: 1,
  204: 1,
  220: 1,
  242: 1,
  418: 1,
  421: 1,
  511: 1,
  529: 1,
  598: 1,
  629: 1,
  667: 1},
 4: {31: 1, 85: 1, 101: 1, 163: 1, 196: 1, 225: 1, 522: 1, 619: 1},
 5: {4: 1,
  11: 2,
  16: 1,
  130: 1,
  221: 1,
  235: 1,
  268: 1,
  362: 1,
  407: 1,
  538: 1,
  554: 1,
  584: 1,
  636: 1,
  697: 1},
 6: {11: 1,
  137: 1,
  235: 1,
  273: 1,
  343: 1,
  345: 1,
  469: 1,
  473: 1,
  508: 1,
  534: 1,
  668: 1,
  702: 1},
 7: {3: 1,
  11: 1,
  14: 1,
  33: 1,
  133: 1,
  152: 1,
  234: 1,
  249: 1,
  311: 2,
  446: 1,
  469: 1,
  476: 1,
  532: 2,
  571: 1,
  579: 1,
  588: 1,
  627: 1,
  640: 2,
  705: 1,
  711: 1,
  715: 2},
 8: {4: 1, 190: 1, 384: 1, 394: 1, 418: 1, 422: 1, 725: 1},
 9: {3: 1,
  16: 1,
  36: 1,
  65: 1,
  106: 1,
  172: 1,
  234: 1,
  254: 1,
  276: 1,
  287: 1,
  407: 1,
  435: 2,
  567: 1,
  

In [92]:
#计算得到各个文章的每个单词的特征权重tf/df
Words_TF_DF = Words_TF
for sentence in Words_TF_DF.values():
    for word in sentence:
        sentence[word] = sentence[word] / Words_DF[word]

In [93]:
Words_TF_DF

{0: {86: 0.1,
  429: 0.058823529411764705,
  446: 0.08695652173913043,
  508: 0.03125},
 1: {99: 0.041666666666666664,
  111: 0.024390243902439025,
  124: 0.030303030303030304,
  196: 0.0037735849056603774,
  296: 0.07692307692307693,
  321: 0.16666666666666666,
  334: 0.07692307692307693,
  627: 0.02857142857142857},
 2: {196: 0.0037735849056603774, 226: 0.04, 278: 0.06060606060606061},
 3: {145: 0.14285714285714285,
  168: 0.125,
  196: 0.0037735849056603774,
  204: 0.0625,
  220: 0.07142857142857142,
  242: 0.1,
  418: 0.018867924528301886,
  421: 0.015384615384615385,
  511: 0.16666666666666666,
  529: 0.022727272727272728,
  598: 0.030303030303030304,
  629: 0.07692307692307693,
  667: 0.043478260869565216},
 4: {31: 0.027777777777777776,
  85: 0.022727272727272728,
  101: 0.08333333333333333,
  163: 0.023809523809523808,
  196: 0.0037735849056603774,
  225: 0.1,
  522: 0.125,
  619: 0.05555555555555555},
 5: {4: 0.08333333333333333,
  11: 0.06666666666666667,
  16: 0.1,
  130: 0.

In [94]:
#用于查询
#list(Words_Dict.keys())[list (Words_Dict.values()).index (653)]

In [95]:
#存入json
import json
with open('Words_Dict.json','w') as f:
        f.write(json.dumps(Words_Dict))
with open('IDs_Dict.json','w') as f:
        f.write(json.dumps(IDs_Dict))
with open('Words_TF_DF.json','w') as f:
        f.write(json.dumps(Words_TF_DF))

In [96]:
#查看字典
with open('Words_Dict.json','r') as f:
    json1=json.load(f)
with open('IDs_Dict.json','r') as f:
    json2=json.load(f)
with open('Words_TF_DF.json','r') as f:
    json3=json.load(f)

In [97]:
json1

{'0': 0,
 '000': 1,
 '1': 2,
 '10': 3,
 '100': 4,
 '12': 5,
 '18': 6,
 '19': 7,
 '2': 8,
 '2014': 9,
 '2015': 10,
 '2016': 11,
 '2018': 12,
 '21': 13,
 '24': 14,
 '271': 15,
 '29': 16,
 '3': 17,
 '30': 18,
 '365': 19,
 '4': 20,
 '5': 21,
 '50': 22,
 '6': 23,
 '7': 24,
 '8': 25,
 '9': 26,
 'abbas': 27,
 'able': 28,
 'accelerate': 29,
 'access': 30,
 'accord': 31,
 'acquire': 32,
 'across': 33,
 'activist': 34,
 'activity': 35,
 'ad': 36,
 'add': 37,
 'agency': 38,
 'ago': 39,
 'agree': 40,
 'agreement': 41,
 'ahead': 42,
 'aim': 43,
 'al': 44,
 'allow': 45,
 'almost': 46,
 'along': 47,
 'already': 48,
 'also': 49,
 'although': 50,
 'amazon': 51,
 'american': 52,
 'amid': 53,
 'amman': 54,
 'among': 55,
 'amount': 56,
 'analyst': 57,
 'android': 58,
 'announce': 59,
 'annual': 60,
 'another': 61,
 'ap': 62,
 'app': 63,
 'appear': 64,
 'apple': 65,
 'application': 66,
 'apps': 67,
 'arabia': 68,
 'area': 69,
 'arm': 70,
 'around': 71,
 'arun': 72,
 'asia': 73,
 'ask': 74,
 'attempt': 75,


In [98]:
json2

{'0': '0',
 '1': '000',
 '10': '2015',
 '100': 'boom',
 '101': 'boost',
 '102': 'brand',
 '103': 'bring',
 '104': 'browser',
 '105': 'budget',
 '106': 'bug',
 '107': 'build',
 '108': 'building',
 '109': 'built',
 '11': '2016',
 '110': 'bureau',
 '111': 'business',
 '112': 'buy',
 '113': 'call',
 '114': 'campaign',
 '115': 'canada',
 '116': 'canadian',
 '117': 'case',
 '118': 'cent',
 '119': 'center',
 '12': '2018',
 '120': 'central',
 '121': 'ceo',
 '122': 'change',
 '123': 'chief',
 '124': 'china',
 '125': 'chinese',
 '126': 'city',
 '127': 'claim',
 '128': 'climate',
 '129': 'cloud',
 '13': '21',
 '130': 'com',
 '131': 'come',
 '132': 'commission',
 '133': 'community',
 '134': 'company',
 '135': 'compute',
 '136': 'computer',
 '137': 'concern',
 '138': 'confidence',
 '139': 'confirm',
 '14': '24',
 '140': 'conflict',
 '141': 'consider',
 '142': 'console',
 '143': 'consumer',
 '144': 'continue',
 '145': 'contract',
 '146': 'cool',
 '147': 'corp',
 '148': 'corporate',
 '149': 'corporat

In [99]:
json3

{'0': {'429': 0.058823529411764705,
  '446': 0.08695652173913043,
  '508': 0.03125,
  '86': 0.1},
 '1': {'111': 0.024390243902439025,
  '124': 0.030303030303030304,
  '196': 0.0037735849056603774,
  '296': 0.07692307692307693,
  '321': 0.16666666666666666,
  '334': 0.07692307692307693,
  '627': 0.02857142857142857,
  '99': 0.041666666666666664},
 '2': {'196': 0.0037735849056603774, '226': 0.04, '278': 0.06060606060606061},
 '3': {'145': 0.14285714285714285,
  '168': 0.125,
  '196': 0.0037735849056603774,
  '204': 0.0625,
  '220': 0.07142857142857142,
  '242': 0.1,
  '418': 0.018867924528301886,
  '421': 0.015384615384615385,
  '511': 0.16666666666666666,
  '529': 0.022727272727272728,
  '598': 0.030303030303030304,
  '629': 0.07692307692307693,
  '667': 0.043478260869565216},
 '4': {'101': 0.08333333333333333,
  '163': 0.023809523809523808,
  '196': 0.0037735849056603774,
  '225': 0.1,
  '31': 0.027777777777777776,
  '522': 0.125,
  '619': 0.05555555555555555,
  '85': 0.022727272727272

In [100]:
len(json1)

729